# Merge fire zones

Join the buildings to "very severe" fire zones mapped by government scientists.

In [1]:
import os
import glob
import geopandas as gpd

In [2]:
%store -r

In [3]:
fire_zones = gpd.read_file(os.path.join(input_dir, "hazard_zones", "very-severe.shp"))

In [5]:
fire_zones.crs

{'init': 'epsg:4326'}

In [4]:
len(fire_zones)

8919

In [73]:
def sjoin_batch(points_path, verbose=False):
    """
    Reads in a batch of buildings and returns then with a column that indicates if they fall within a fire zone.
    """
    # Set the path
    number = points_path.split("-batch-")[1].replace(".shp", "")
    outpath = os.path.join(output_dir, f"california-building-points-batch-{number}-to-fire-zones.csv")

    # Skip if it already exists
    if os.path.exists(outpath):
        if verbose:
            print(f"Skipping batch {number}")
        return
    
    # Read in the file
    points = gpd.read_file(points_path)
    
    # Run a spatial join against the fire zones
    sjoin = gpd.sjoin(points, fire_zones, how="inner", op='intersects')
    
    # Mark the ones that are in a fire zone
    points['in_fire_zone'] = points.FID.isin(sjoin.FID)
    points['in_lra_zone'] = points.FID.isin(sjoin[sjoin.SRA == 'LRA'].FID)
    points['in_sra_zone'] = points.FID.isin(sjoin[sjoin.SRA == 'SRA'].FID)
    
    def create_combined_field(row):
        if row.in_lra_zone:
            return 'LRA'
        elif row.in_sra_zone:
            return 'SRA'
        else:
            return gpd.pd.np.NaN
    points['fire_zone_type'] = points.apply(create_combined_field, axis=1)

    # Log the hits, if you've been asked to.
    if verbose:
        hits = points[points['in_fire_zone'] == True]
        print(f"{len(hits):,}/{len(points):,} hits in {points_path}.")
    
    # Save the file
    if verbose:
        print(f"Creating {outpath}")
    trimmed_points = points[[
        'FID',
        'fire_zone_type'
    ]]
    trimmed_points.to_csv(outpath, index=False)

In [74]:
path_list = sorted(
    glob.glob(os.path.join(output_dir, "california-building-points-batch-*.shp")),
    key=lambda x: int(x.split("-batch-")[1].replace(".shp", ""))
)

In [75]:
len(path_list)

1099

In [76]:
sjoin_batch(path_list[0], verbose=True)

Skipping batch 1


In [78]:
for p in path_list: sjoin_batch(p)

In [81]:
csv_list = glob.glob(os.path.join(output_dir, "california-building-points-batch-*-to-fire-zones.csv"))

In [82]:
csv_df = gpd.pd.concat([gpd.pd.read_csv(f) for f in csv_list])

In [84]:
len(csv_df)

10988525

In [85]:
csv_df.to_csv(os.path.join(output_dir, "california-building-points-to-fire-zones.csv"), index=False)